# ⚽ Previsão Avançada de Resultados de Futebol

Este notebook combina técnicas de Machine Learning para prever resultados de jogos de futebol com maior precisão.

## Inclui:
- Dados históricos (Premier League)
- Cálculo da forma recente dos times
- Uso de probabilidades das casas de apostas (odds)
- Modelo XGBoost para previsão
- Previsão de uma rodada futura baseada nos dados anteriores


In [ ]:
# 📦 Instalar bibliotecas necessárias
# !pip install pandas xgboost scikit-learn matplotlib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
import matplotlib.pyplot as plt

In [ ]:
# 📥 Carregar dados da Premier League 2022/2023 com odds
url = 'https://www.football-data.co.uk/mmz4281/2223/E0.csv'
df = pd.read_csv(url)
df = df[['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR','B365H','B365D','B365A']]
df.columns = ['Date','HomeTeam','AwayTeam','HomeGoals','AwayGoals','Result','OddsH','OddsD','OddsA']
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df = df.sort_values('Date')
df.reset_index(drop=True, inplace=True)
df.head()

In [ ]:
# 🔁 Calcular forma recente (últimos 5 jogos)
def calcular_forma(time, idx, coluna_gols, df_filtrado):
    ultimos = df_filtrado.loc[:idx-1]
    jogos_time = ultimos[(ultimos['HomeTeam'] == time) | (ultimos['AwayTeam'] == time)].tail(5)
    if jogos_time.empty:
        return 0
    pontos = 0
    for _, jogo in jogos_time.iterrows():
        if jogo['HomeTeam'] == time:
            if jogo['HomeGoals'] > jogo['AwayGoals']:
                pontos += 3
            elif jogo['HomeGoals'] == jogo['AwayGoals']:
                pontos += 1
        elif jogo['AwayTeam'] == time:
            if jogo['AwayGoals'] > jogo['HomeGoals']:
                pontos += 3
            elif jogo['AwayGoals'] == jogo['HomeGoals']:
                pontos += 1
    return pontos / 15

df['HomeForm'] = 0.0
df['AwayForm'] = 0.0
for i in range(len(df)):
    df.at[i, 'HomeForm'] = calcular_forma(df.at[i, 'HomeTeam'], i, 'HomeGoals', df)
    df.at[i, 'AwayForm'] = calcular_forma(df.at[i, 'AwayTeam'], i, 'AwayGoals', df)
df[['HomeTeam','AwayTeam','HomeForm','AwayForm']].head(10)

In [ ]:
# 🧠 Codificar nomes de times para números
teams = pd.concat([df['HomeTeam'], df['AwayTeam']]).unique()
team_map = {team: idx for idx, team in enumerate(teams)}
df['HomeTeam'] = df['HomeTeam'].map(team_map)
df['AwayTeam'] = df['AwayTeam'].map(team_map)

In [ ]:
# 🔢 Preparar dados para o modelo XGBoost
X = df[['HomeTeam','AwayTeam','HomeForm','AwayForm','OddsH','OddsD','OddsA']]
y = df['Result']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)
preds = model.predict(X_test)
acc = accuracy_score(y_test, preds)
print(f"Acurácia: {acc * 100:.2f}%")

In [ ]:
# 📊 Avaliar desempenho do modelo
print(classification_report(y_test, preds))